In [1]:
import pandas as pd
import numpy as np

In [2]:
#!curl -L https://www.dropbox.com/s/4hct840aqr9drpl/mturk_surveys.csv?dl=1 -o mturk_surveys.csv


df = pd.read_csv("mturk_surveys.csv")
df["hit_answered_date"]=pd.to_datetime(df["hit_answered_date"])
df['date'] = pd.to_datetime(df['hit_answered_date'].apply(lambda x : x.date()))

# Create a list of all unique worker IDs 
workers = pd.DataFrame(list(set(df.worker_id)), columns=['worker_id']).reset_index().set_index('worker_id')
I = len(workers)

# Calculate the time period of each survey answer. 
# Below we take out time period to be 30-day periods. 
# We can change this to be weeks, months, or anything else
window_length = 30
minDate = min(df['date'])
df['period'] = (df['date'] - minDate).dt.days // window_length 

K = df['period'].max() + 1

data = df.pivot_table(
    index = 'worker_id',
    values = 'period',                    
    aggfunc=('count','min','max')
).join(workers).set_index('index').sort_index()

data.head(10)

,count,max,min
index,,,
0,1,3,3
1,1,17,17
2,10,24,13
3,7,26,8
4,2,25,6
5,1,16,16
6,1,15,15
7,1,0,0
8,1,15,15


In [6]:
data[data['count'] > (data['max']-data['min'] + 1)].head(10)

,count,max,min
index,,,
33,3,34,33
228,3,27,27
253,2,12,12
297,2,30,30
351,3,3,2
396,2,15,15
426,2,9,9
428,2,30,30
513,2,33,33
